# ДЗ по "Введение в нейронные сети (Вебинар)"

## Вебинар 5. Рекуррентные нейронные сети 
* Домашнее задание:  
1. Попробуйте починить сеть по словам.  
2. Попробуйте изменить параметры нейронной сети, генерирующей текст таким образом, чтобы добиться генерации как можно 
более осмысленного текста. Пришлите лучший текст из получившихся и опишите предпринятые для его получения действия. 
Можно использовать текст другого произведения  

Сдавать как обычно гитхабом или гугл колабом.

In [1]:
# Устанавливаем пакеты в окружение, если они не установлены
# !pip install scikeras[tensorflow]

In [2]:
import re
import keras
from keras.preprocessing.text import Tokenizer
import numpy as np
from keras.models import Sequential
from keras.layers import Input, SimpleRNN, Dense
from keras.callbacks import EarlyStopping
from keras.utils import to_categorical

In [3]:
# Константы
EPOCH = 100 # Количество эпох в модели обучения

## Прогнозируем символы  

ссылка на материал - https://proproprogs.ru/neural_network/kak-rekurrentnaya-neyronnaya-set-prognoziruet-simvoly

In [4]:
# И, чтобы было проще решать эту задачу, оставим в тексте только символы русских букв и символы пробела:
with open('train_data.txt', 'r', encoding='utf-8') as f:
    text = f.read()
    text = text.replace('\ufeff', '')   # убираем первый невидимый символ
    text = re.sub(r'[^А-я ]', '', text) # убираем все недопустимые символы

In [5]:
text

'Вы  лучший ответ на проблемы которые возникли в понедельникДумайте позитивно и верьте в свою способность достигать отличных результатовЕсли вы смогли в понедельник подняться с постели значит вы супер герой'

In [6]:
# То есть, всего у нас будет 34 разных символа:
num_characters = 34  # 33 буквы + пробел

In [7]:
# Чтобы им воспользоваться, сначала нужно создать экземпляр класса Tokenizer, который имеет следующие важные параметры:

# num_words – максимальное количество слов (символов), которое вернет Tokenizer (если элементов будет больше, 
# то останутся наиболее повторяющиеся в тексте);
# filters – исключаемые из текста символы (по умолчанию, следующие: !–»—#$%&()*+,-./:;<=>?@[\]^_`{|}~\t\n\r);
# lower = True – автоматический перевод в нижний регистр для единообразия больших и малых символов;
# split = '  ' – разделение слов по символу пробела;
# char_level=False – если False, то текст делится на слова, иначе – на символы.

tokenizer = Tokenizer(num_words=num_characters, char_level=True)

In [8]:
# И пропустим через него загруженный текст
tokenizer.fit_on_texts(text)

In [9]:
# В итоге, формируется словарь:
print(tokenizer.word_index)

{' ': 1, 'о': 2, 'т': 3, 'е': 4, 'и': 5, 'в': 6, 'н': 7, 'с': 8, 'л': 9, 'п': 10, 'ь': 11, 'ы': 12, 'р': 13, 'а': 14, 'д': 15, 'у': 16, 'к': 17, 'з': 18, 'ч': 19, 'й': 20, 'м': 21, 'г': 22, 'б': 23, 'я': 24, 'ш': 25, 'ю': 26, 'х': 27}


In [10]:
# Далее, преобразуем текст в набор OHE-векторов:
inp_chars = 4 # Число первых символов
data = tokenizer.texts_to_matrix(text)

In [11]:
data[1]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [12]:
# Для начала вычислим размер обучающего множества:
n = data.shape[0]-inp_chars
n  #размер обучающего множества

201

In [13]:
# И, далее, сформируем входной тензор и прогнозные значения:
X = np.array([data[i:i+inp_chars, :] for i in range(n)])
Y = data[inp_chars:] #предсказание следующего символа

In [14]:
# Отлично, данные для обучения готовы. Теперь создадим рекуррентную НС с помощью Keras
model = Sequential()
model.add(Input((inp_chars, num_characters)))
model.add(SimpleRNN(500, activation='tanh'))
model.add(Dense(num_characters, activation='softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn (SimpleRNN)      (None, 500)               267500    
                                                                 
 dense (Dense)               (None, 34)                17034     
                                                                 
Total params: 284,534
Trainable params: 284,534
Non-trainable params: 0
_________________________________________________________________


### Создаём callback EarlyStopping (защита от переобучения)
см. [ссылка ToTube](https://www.youtube.com/watch?v=aiH_oXtuzjE) 

In [15]:
# Настраиваем защиту от переобучения
erly_stoping_callback = EarlyStopping(monitor='accuracy', patience=3)  # Если метрика 'val_accuracy' продолжает снижение два шага подряд,
                                                                           # то выходим из процесса обучения

In [16]:
# Компилируем и обучаем модель НС
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')
history = model.fit(X, Y, batch_size=32, epochs=EPOCH,  callbacks=[erly_stoping_callback])

Epoch 1/100
7/7 [==============================] - 1s 5ms/step - loss: 3.4506 - accuracy: 0.0597
Epoch 2/100
7/7 [==============================] - 0s 4ms/step - loss: 2.6368 - accuracy: 0.2886
Epoch 3/100
7/7 [==============================] - 0s 4ms/step - loss: 2.2354 - accuracy: 0.3532
Epoch 4/100
7/7 [==============================] - 0s 4ms/step - loss: 1.9478 - accuracy: 0.4677
Epoch 5/100
7/7 [==============================] - 0s 4ms/step - loss: 1.6363 - accuracy: 0.5025
Epoch 6/100
7/7 [==============================] - 0s 4ms/step - loss: 1.4219 - accuracy: 0.5473
Epoch 7/100
7/7 [==============================] - 0s 4ms/step - loss: 1.2738 - accuracy: 0.5970
Epoch 8/100
7/7 [==============================] - 0s 4ms/step - loss: 1.1126 - accuracy: 0.6965
Epoch 9/100
7/7 [==============================] - 0s 4ms/step - loss: 1.0850 - accuracy: 0.6667
Epoch 10/100
7/7 [==============================] - 0s 6ms/step - loss: 0.9354 - accuracy: 0.6965
Epoch 11/100
7/7 [===========

In [17]:
if erly_stoping_callback.stopped_epoch >=1:
    print('Обучение остановлено на эпохе', erly_stoping_callback.stopped_epoch)

Обучение остановлено на эпохе 24


In [18]:
# Далее, объявим вспомогательную функцию, в которой будет выполняться прогноз очередного символа и добавления его в конец начальной строки:
def buildPhrase(inp_str, str_len = 50):
  for i in range(str_len):
    x = []
    for j in range(i, i+inp_chars):
      x.append(tokenizer.texts_to_matrix(inp_str[j])) # преобразуем символы в One-Hot-encoding

    x = np.array(x)
    inp = x.reshape(1, inp_chars, num_characters)
    print(f'{inp_chars = }')
    print(f'{num_characters = }')
    print(f'{inp.shape = }')
    

    pred = model.predict( inp ) # предсказываем OHE четвертого символа
    d = tokenizer.index_word[pred.argmax(axis=1)[0]] # получаем ответ в символьном представлении

    inp_str += d # дописываем строку

  return inp_str

In [19]:
# Ей на вход подается начальная строка длиной inp_chars символов и, затем, она подается на НС и прогнозируется следующий символ. Далее, 
# мы используем это прогнозное значение и получаем следующее и так далее делаем str_len раз.
res = buildPhrase("лучш")

inp_chars = 4
num_characters = 34
inp.shape = (1, 4, 34)
1/1 [==============================] - 0s 86ms/step
inp_chars = 4
num_characters = 34
inp.shape = (1, 4, 34)
1/1 [==============================] - 0s 11ms/step
inp_chars = 4
num_characters = 34
inp.shape = (1, 4, 34)
1/1 [==============================] - 0s 8ms/step
inp_chars = 4
num_characters = 34
inp.shape = (1, 4, 34)
1/1 [==============================] - 0s 14ms/step
inp_chars = 4
num_characters = 34
inp.shape = (1, 4, 34)
1/1 [==============================] - 0s 14ms/step
inp_chars = 4
num_characters = 34
inp.shape = (1, 4, 34)
1/1 [==============================] - 0s 13ms/step
inp_chars = 4
num_characters = 34
inp.shape = (1, 4, 34)
1/1 [==============================] - 0s 14ms/step
inp_chars = 4
num_characters = 34
inp.shape = (1, 4, 34)
1/1 [==============================] - 0s 13ms/step
inp_chars = 4
num_characters = 34
inp.shape = (1, 4, 34)
1/1 [==============================] - 0s 14ms/step
inp_chars = 4
num_ch

In [20]:
print(res)

лучший отвеч извеьликвыорле в свою способломыикотнрче 


## Делаем прогноз слов рекуррентной сетью Embedding слой

Ссылка на материал - https://proproprogs.ru/neural_network/delaem-prognoz-slov-rekurrentnoy-setyu-embedding-sloy

In [21]:
# Загрузим текст
with open('train_data.txt', 'r', encoding='utf-8') as f:
    texts = f.read()
    texts = texts.replace('\ufeff', '') # убираем первый невидимый симво

In [22]:
# Теперь нам нужно разбить эти высказывания на слова. Для этого воспользуемся уже знакомым из прошлого занятия инструментом Tokenizer и 
# положим, что максимальное число слов будет равно 1000:
maxWordsCount = 1000
tokenizer = Tokenizer(num_words=maxWordsCount, filters='!–"—#$%&amp;()*+,-./:;<=>?@[\\]^_`{|}~\t\n\r«»',
                       lower=True, split=' ', char_level=False)
tokenizer.fit_on_texts([texts])

In [23]:
# Итак, мы разбили текст на слова и для примера выведем их начальный список:
dist = list(tokenizer.word_counts.items())
print(dist[:10])

[('вы', 3), ('лучший', 1), ('ответ', 1), ('на', 1), ('проблемы', 1), ('которые', 1), ('возникли', 1), ('в', 3), ('понедельник', 2), ('думайте', 1)]


In [24]:
# Далее, мы преобразуем текст в последовательность чисел в соответствии с полученным словарем. 
# Для этого используется специальный метод класса Tokenizer:
data = tokenizer.texts_to_sequences([texts])

In [25]:
# Осталось закодировать числа массива data в one-hot векторы. Для этого мы воспользуемся методом to_categorical пакета Keras:
res = to_categorical(data[0], num_classes=maxWordsCount)
print( res.shape )

(31, 1000)


In [26]:
# Затем, из этой матрицы сформируем тензор обучающей выборки и соответствующий набор выходных значений. 
# Для начала вычислим размер обучающего множества:
inp_words = 3
n = res.shape[0]-inp_words

In [27]:
# И, далее, сформируем входной тензор и прогнозные значения также, как мы это делали с символами:
X = np.array([res[i:i+inp_words, :] for i in range(n)])
Y = res[inp_words:]

In [28]:
# Все, у нас есть обучающая выборка и требуемые выходные значения. Осталось создать модель рекуррентной сети. 
# Мы ее возьмем из предыдущего занятия с числом нейронов скрытого слоя 128 и maxWordsCount нейронами на выходе с функцией активации softmax:
model_2 = Sequential()
model_2.add(Input((inp_words, maxWordsCount)))
model_2.add(SimpleRNN(128, activation='tanh'))
model_2.add(Dense(maxWordsCount, activation='softmax'))
model_2.summary()
 
model_2.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn_1 (SimpleRNN)    (None, 128)               144512    
                                                                 
 dense_1 (Dense)             (None, 1000)              129000    
                                                                 
Total params: 273,512
Trainable params: 273,512
Non-trainable params: 0
_________________________________________________________________


In [29]:
# Готово. Запускаем процесс обучения:
history_2 = model_2.fit(X, Y, batch_size=32, epochs=EPOCH, callbacks=[erly_stoping_callback])

Epoch 1/100
1/1 [==============================] - 0s 473ms/step - loss: 6.9059 - accuracy: 0.0000e+00
Epoch 2/100
1/1 [==============================] - 0s 6ms/step - loss: 6.8802 - accuracy: 0.0000e+00
Epoch 3/100
1/1 [==============================] - 0s 8ms/step - loss: 6.8543 - accuracy: 0.0714
Epoch 4/100
1/1 [==============================] - 0s 6ms/step - loss: 6.8280 - accuracy: 0.1429
Epoch 5/100
1/1 [==============================] - 0s 6ms/step - loss: 6.8010 - accuracy: 0.4643
Epoch 6/100
1/1 [==============================] - 0s 83ms/step - loss: 6.7732 - accuracy: 0.5714
Epoch 7/100
1/1 [==============================] - 0s 6ms/step - loss: 6.7441 - accuracy: 0.8214
Epoch 8/100
1/1 [==============================] - 0s 6ms/step - loss: 6.7134 - accuracy: 0.9286
Epoch 9/100
1/1 [==============================] - 0s 6ms/step - loss: 6.6809 - accuracy: 0.9643
Epoch 10/100
1/1 [==============================] - 0s 6ms/step - loss: 6.6461 - accuracy: 0.9643
Epoch 11/100
1/1 [

In [30]:
# Смотрим на какой эпохе остановилось обучение
if erly_stoping_callback.stopped_epoch >=1:
    print('Обучение остановлено на эпохе', erly_stoping_callback.stopped_epoch)

Обучение остановлено на эпохе 11


In [31]:
# И давайте теперь посмотрим, что у нас получилось. Запишем функцию для формирования текста из спрогнозированных слов:
def buildPhrase_2(texts, str_len = 20):
  res = texts
  data = tokenizer.texts_to_sequences([texts])[0]
  print(f'{len(data) = }')
  for i in range(str_len):
    x = to_categorical(data[i: i+inp_words], num_classes=maxWordsCount) # преобразуем в One-Hot-encoding
    print(f'{x.shape = }')
    inp = x.reshape(1, inp_words, maxWordsCount)
 
    pred = model_2.predict( inp ) # предсказываем OHE четвертого символа
    indx = pred.argmax(axis=1)[0]
    data.append(indx)
 
    res += " " + tokenizer.index_word[indx] # дописываем строку
 
  return res

In [32]:
# И вызовем ее с тремя первыми словами
res = buildPhrase_2('лучший ответ на')

len(data) = 3
x.shape = (3, 1000)
1/1 [==============================] - 0s 81ms/step
x.shape = (3, 1000)
1/1 [==============================] - 0s 12ms/step
x.shape = (3, 1000)
1/1 [==============================] - 0s 12ms/step
x.shape = (3, 1000)
1/1 [==============================] - 0s 14ms/step
x.shape = (3, 1000)
1/1 [==============================] - 0s 12ms/step
x.shape = (3, 1000)
1/1 [==============================] - 0s 13ms/step
x.shape = (3, 1000)
1/1 [==============================] - 0s 12ms/step
x.shape = (3, 1000)
1/1 [==============================] - 0s 12ms/step
x.shape = (3, 1000)
1/1 [==============================] - 0s 12ms/step
x.shape = (3, 1000)
1/1 [==============================] - 0s 14ms/step
x.shape = (3, 1000)
1/1 [==============================] - 0s 12ms/step
x.shape = (3, 1000)
1/1 [==============================] - 0s 13ms/step
x.shape = (3, 1000)
1/1 [==============================] - 0s 11ms/step
x.shape = (3, 1000)
1/1 [=========================

In [33]:
print(res)

лучший ответ на проблемы которые возникли в понедельник думайте позитивно и верьте в свою способность достигать отличных результатов если вы смогли в понедельник


## Вывод:
### Выполнив данное ДЗ, на практике посмотрел как предсказываются последующие буквы и слова с помощью рекурентных НС.